In [1]:
import numpy as np
import pandas as pd

In [2]:
####version 3 of dataframe,same as 2 but encoding late a quartile, aslso including arr_dely

In [3]:
from sqlalchemy import create_engine
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'lhl_student' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'lhl_student' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD
POSTGRES_DBNAME = 'mid_term_project' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
full = pd.read_sql_query('''DROP TABLE IF EXISTS flight_subset;
DROP TABLE IF EXISTS passenger_subset;

SELECT AVG(arr_delay) AS avg_arr_delay, op_unique_carrier, AVG(dep_delay) AS avg_dep_delay, CONCAT(flights.origin_airport_id, '-', flights.dest_airport_id) AS route, SUBSTRING(fl_date, 6, 2) AS month, SUBSTRING(fl_date, 1, 4) AS year
INTO TABLE flight_subset
FROM flights
WHERE diverted != 1.0
GROUP BY route, month, year, op_unique_carrier;

ALTER TABLE flight_subset ALTER COLUMN month TYPE BIGINT USING(month::bigint), ALTER COLUMN year TYPE BIGINT USING (year::bigint);

SELECT departures_scheduled, departures_performed, seats, passengers, freight, mail, aircraft_group, aircraft_type, unique_carrier, region, origin_airport_id, dest_airport_id, CONCAT(origin_airport_id, '-', dest_airport_id) AS route, month, year, distance
INTO TABLE passenger_subset
FROM passengers;

SELECT MAX(avg_arr_delay) AS avg_arr_delay, passenger_subset.route, passenger_subset.month, passenger_subset.year, unique_carrier, MAX(avg_dep_delay) AS avg_dep_delay, MAX(departures_scheduled) AS departures_scheduled, MAX(departures_performed) AS departures_performed, MAX(seats) AS seats, MAX(passengers) AS passengers, MAX(freight) AS freight, MAX(mail) AS mail, MAX(aircraft_group) AS aircraft_group, MAX(aircraft_type) AS aircraft_type, MAX(region) AS region, MAX(passenger_subset.origin_airport_id) AS origin_airport_id, MAX(passenger_subset.dest_airport_id) AS dest_airport_id, MAX(distance) AS distance
FROM flight_subset JOIN passenger_subset ON (flight_subset.route = passenger_subset.route AND op_unique_carrier = unique_carrier AND flight_subset.month = passenger_subset.month AND flight_subset.year = passenger_subset.year)
GROUP BY passenger_subset.route, passenger_subset.month, passenger_subset.year, unique_carrier;''',cnx)

In [5]:
full

,avg_arr_delay,route,month,year,unique_carrier,avg_dep_delay,departures_scheduled,departures_performed,seats,passengers,freight,mail,aircraft_group,aircraft_type,region,origin_airport_id,dest_airport_id,distance
0,13.206897,10135-10397,1,2018,DL,19.965517,29.0,29.0,3190.0,2475.0,0.0,0.0,6,608,D,10135,10397,692.0
1,-3.000000,10135-10397,1,2018,EV,3.000000,36.0,35.0,2660.0,1998.0,0.0,0.0,6,638,D,10135,10397,692.0
2,7.073171,10135-10397,1,2019,9E,9.365854,41.0,40.0,3040.0,2492.0,0.0,0.0,6,638,D,10135,10397,692.0
3,-0.923077,10135-10397,1,2019,DL,3.423077,26.0,26.0,2860.0,2422.0,71.0,0.0,6,608,D,10135,10397,692.0
4,5.500000,10135-10397,2,2018,9E,8.600000,16.0,16.0,1216.0,980.0,0.0,0.0,6,638,D,10135,10397,692.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258626,60.647887,16869-11292,12,2019,AX,60.169014,73.0,71.0,3550.0,3335.0,0.0,0.0,6,675,D,16869,11292,582.0
258627,21.055556,16869-11292,12,2019,OO,34.789474,20.0,19.0,1444.0,1255.0,0.0,0.0,6,673,D,16869,11292,582.0
258628,8.325581,16869-13487,10,2019,OO,14.674419,43.0,43.0,2150.0,1951.0,0.0,0.0,6,629,D,16869,13487,553.0
258629,8.396552,16869-13487,11,2019,OO,22.051724,31.0,31.0,1794.0,1598.0,0.0,0.0,6,638,D,16869,13487,553.0


In [6]:
full = full.dropna()

In [7]:
full

,avg_arr_delay,route,month,year,unique_carrier,avg_dep_delay,departures_scheduled,departures_performed,seats,passengers,freight,mail,aircraft_group,aircraft_type,region,origin_airport_id,dest_airport_id,distance
0,13.206897,10135-10397,1,2018,DL,19.965517,29.0,29.0,3190.0,2475.0,0.0,0.0,6,608,D,10135,10397,692.0
1,-3.000000,10135-10397,1,2018,EV,3.000000,36.0,35.0,2660.0,1998.0,0.0,0.0,6,638,D,10135,10397,692.0
2,7.073171,10135-10397,1,2019,9E,9.365854,41.0,40.0,3040.0,2492.0,0.0,0.0,6,638,D,10135,10397,692.0
3,-0.923077,10135-10397,1,2019,DL,3.423077,26.0,26.0,2860.0,2422.0,71.0,0.0,6,608,D,10135,10397,692.0
4,5.500000,10135-10397,2,2018,9E,8.600000,16.0,16.0,1216.0,980.0,0.0,0.0,6,638,D,10135,10397,692.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258626,60.647887,16869-11292,12,2019,AX,60.169014,73.0,71.0,3550.0,3335.0,0.0,0.0,6,675,D,16869,11292,582.0
258627,21.055556,16869-11292,12,2019,OO,34.789474,20.0,19.0,1444.0,1255.0,0.0,0.0,6,673,D,16869,11292,582.0
258628,8.325581,16869-13487,10,2019,OO,14.674419,43.0,43.0,2150.0,1951.0,0.0,0.0,6,629,D,16869,13487,553.0
258629,8.396552,16869-13487,11,2019,OO,22.051724,31.0,31.0,1794.0,1598.0,0.0,0.0,6,638,D,16869,13487,553.0


In [8]:
full.nunique()

avg_arr_delay           94663
route                    6938
month                      12
year                        2
unique_carrier             28
avg_dep_delay           88839
departures_scheduled      557
departures_performed      561
seats                   17124
passengers              24811
freight                 30976
mail                    22857
aircraft_group              3
aircraft_type              44
region                      3
origin_airport_id         376
dest_airport_id           376
distance                 1613
dtype: int64

In [9]:
full["avg_dep_delay"].head()

0    19.965517
1     3.000000
2     9.365854
3     3.423077
4     8.600000
Name: avg_dep_delay, dtype: float64

In [10]:
full

,avg_arr_delay,route,month,year,unique_carrier,avg_dep_delay,departures_scheduled,departures_performed,seats,passengers,freight,mail,aircraft_group,aircraft_type,region,origin_airport_id,dest_airport_id,distance
0,13.206897,10135-10397,1,2018,DL,19.965517,29.0,29.0,3190.0,2475.0,0.0,0.0,6,608,D,10135,10397,692.0
1,-3.000000,10135-10397,1,2018,EV,3.000000,36.0,35.0,2660.0,1998.0,0.0,0.0,6,638,D,10135,10397,692.0
2,7.073171,10135-10397,1,2019,9E,9.365854,41.0,40.0,3040.0,2492.0,0.0,0.0,6,638,D,10135,10397,692.0
3,-0.923077,10135-10397,1,2019,DL,3.423077,26.0,26.0,2860.0,2422.0,71.0,0.0,6,608,D,10135,10397,692.0
4,5.500000,10135-10397,2,2018,9E,8.600000,16.0,16.0,1216.0,980.0,0.0,0.0,6,638,D,10135,10397,692.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258626,60.647887,16869-11292,12,2019,AX,60.169014,73.0,71.0,3550.0,3335.0,0.0,0.0,6,675,D,16869,11292,582.0
258627,21.055556,16869-11292,12,2019,OO,34.789474,20.0,19.0,1444.0,1255.0,0.0,0.0,6,673,D,16869,11292,582.0
258628,8.325581,16869-13487,10,2019,OO,14.674419,43.0,43.0,2150.0,1951.0,0.0,0.0,6,629,D,16869,13487,553.0
258629,8.396552,16869-13487,11,2019,OO,22.051724,31.0,31.0,1794.0,1598.0,0.0,0.0,6,638,D,16869,13487,553.0


In [11]:
len(full["avg_arr_delay"])

258587

In [12]:
full = full.reset_index()

In [13]:
delay = []
for i in range(len(full["avg_arr_delay"])):
    if full["avg_arr_delay"][i] > 0:
        delay.append(1)
    if full["avg_arr_delay"][i] <= 0:
        delay.append(0)

In [36]:
full["avg_arr_delay"] = delay

In [15]:
full

,index,avg_arr_delay,route,month,year,unique_carrier,avg_dep_delay,departures_scheduled,departures_performed,seats,passengers,freight,mail,aircraft_group,aircraft_type,region,origin_airport_id,dest_airport_id,distance
0,0,1,10135-10397,1,2018,DL,19.965517,29.0,29.0,3190.0,2475.0,0.0,0.0,6,608,D,10135,10397,692.0
1,1,0,10135-10397,1,2018,EV,3.000000,36.0,35.0,2660.0,1998.0,0.0,0.0,6,638,D,10135,10397,692.0
2,2,1,10135-10397,1,2019,9E,9.365854,41.0,40.0,3040.0,2492.0,0.0,0.0,6,638,D,10135,10397,692.0
3,3,0,10135-10397,1,2019,DL,3.423077,26.0,26.0,2860.0,2422.0,71.0,0.0,6,608,D,10135,10397,692.0
4,4,1,10135-10397,2,2018,9E,8.600000,16.0,16.0,1216.0,980.0,0.0,0.0,6,638,D,10135,10397,692.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258582,258626,1,16869-11292,12,2019,AX,60.169014,73.0,71.0,3550.0,3335.0,0.0,0.0,6,675,D,16869,11292,582.0
258583,258627,1,16869-11292,12,2019,OO,34.789474,20.0,19.0,1444.0,1255.0,0.0,0.0,6,673,D,16869,11292,582.0
258584,258628,1,16869-13487,10,2019,OO,14.674419,43.0,43.0,2150.0,1951.0,0.0,0.0,6,629,D,16869,13487,553.0
258585,258629,1,16869-13487,11,2019,OO,22.051724,31.0,31.0,1794.0,1598.0,0.0,0.0,6,638,D,16869,13487,553.0


In [16]:
full

,index,avg_arr_delay,route,month,year,unique_carrier,avg_dep_delay,departures_scheduled,departures_performed,seats,passengers,freight,mail,aircraft_group,aircraft_type,region,origin_airport_id,dest_airport_id,distance
0,0,1,10135-10397,1,2018,DL,19.965517,29.0,29.0,3190.0,2475.0,0.0,0.0,6,608,D,10135,10397,692.0
1,1,0,10135-10397,1,2018,EV,3.000000,36.0,35.0,2660.0,1998.0,0.0,0.0,6,638,D,10135,10397,692.0
2,2,1,10135-10397,1,2019,9E,9.365854,41.0,40.0,3040.0,2492.0,0.0,0.0,6,638,D,10135,10397,692.0
3,3,0,10135-10397,1,2019,DL,3.423077,26.0,26.0,2860.0,2422.0,71.0,0.0,6,608,D,10135,10397,692.0
4,4,1,10135-10397,2,2018,9E,8.600000,16.0,16.0,1216.0,980.0,0.0,0.0,6,638,D,10135,10397,692.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258582,258626,1,16869-11292,12,2019,AX,60.169014,73.0,71.0,3550.0,3335.0,0.0,0.0,6,675,D,16869,11292,582.0
258583,258627,1,16869-11292,12,2019,OO,34.789474,20.0,19.0,1444.0,1255.0,0.0,0.0,6,673,D,16869,11292,582.0
258584,258628,1,16869-13487,10,2019,OO,14.674419,43.0,43.0,2150.0,1951.0,0.0,0.0,6,629,D,16869,13487,553.0
258585,258629,1,16869-13487,11,2019,OO,22.051724,31.0,31.0,1794.0,1598.0,0.0,0.0,6,638,D,16869,13487,553.0


In [17]:
full.isnull().nunique()

index                   1
avg_arr_delay           1
route                   1
month                   1
year                    1
unique_carrier          1
avg_dep_delay           1
departures_scheduled    1
departures_performed    1
seats                   1
passengers              1
freight                 1
mail                    1
aircraft_group          1
aircraft_type           1
region                  1
origin_airport_id       1
dest_airport_id         1
distance                1
dtype: int64

In [18]:
mo = pd.get_dummies(full["month"], prefix = "Month")

In [19]:
ye =pd.get_dummies(full["year"], prefix = "year")

In [20]:
opa = pd.get_dummies(full["unique_carrier"], prefix = "unique_carrier")

In [21]:
ag = pd.get_dummies(full["aircraft_group"], prefix = "aircraft_group")

In [22]:
at = pd.get_dummies(full["aircraft_type"], prefix = "aircraft_type")

In [23]:
re = pd.get_dummies(full["region"], prefix = "Region")

In [24]:
oai = pd.get_dummies(full["origin_airport_id"], prefix = "origin_airport_id")

In [25]:
dai = pd.get_dummies(full["dest_airport_id"], prefix = "dest_airport_id")

In [26]:
full = pd.concat([full,mo,ye,opa,ag,at,re,oai,dai], axis=1)

In [27]:
full

,index,avg_arr_delay,route,month,year,unique_carrier,avg_dep_delay,departures_scheduled,departures_performed,seats,...,dest_airport_id_15607,dest_airport_id_15624,dest_airport_id_15841,dest_airport_id_15897,dest_airport_id_15919,dest_airport_id_15991,dest_airport_id_16101,dest_airport_id_16133,dest_airport_id_16218,dest_airport_id_16869
0,0,1,10135-10397,1,2018,DL,19.965517,29.0,29.0,3190.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,10135-10397,1,2018,EV,3.000000,36.0,35.0,2660.0,...,0,0,0,0,0,0,0,0,0,0
2,2,1,10135-10397,1,2019,9E,9.365854,41.0,40.0,3040.0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,10135-10397,1,2019,DL,3.423077,26.0,26.0,2860.0,...,0,0,0,0,0,0,0,0,0,0
4,4,1,10135-10397,2,2018,9E,8.600000,16.0,16.0,1216.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258582,258626,1,16869-11292,12,2019,AX,60.169014,73.0,71.0,3550.0,...,0,0,0,0,0,0,0,0,0,0
258583,258627,1,16869-11292,12,2019,OO,34.789474,20.0,19.0,1444.0,...,0,0,0,0,0,0,0,0,0,0
258584,258628,1,16869-13487,10,2019,OO,14.674419,43.0,43.0,2150.0,...,0,0,0,0,0,0,0,0,0,0
258585,258629,1,16869-13487,11,2019,OO,22.051724,31.0,31.0,1794.0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
full = full.drop(columns=["index","avg_dep_delay","month","year","route", "unique_carrier", "aircraft_group","aircraft_type", "region", "origin_airport_id","dest_airport_id"])

In [29]:
full

,avg_arr_delay,departures_scheduled,departures_performed,seats,passengers,freight,mail,distance,Month_1,Month_2,...,dest_airport_id_15607,dest_airport_id_15624,dest_airport_id_15841,dest_airport_id_15897,dest_airport_id_15919,dest_airport_id_15991,dest_airport_id_16101,dest_airport_id_16133,dest_airport_id_16218,dest_airport_id_16869
0,1,29.0,29.0,3190.0,2475.0,0.0,0.0,692.0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,36.0,35.0,2660.0,1998.0,0.0,0.0,692.0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,41.0,40.0,3040.0,2492.0,0.0,0.0,692.0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,26.0,26.0,2860.0,2422.0,71.0,0.0,692.0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,16.0,16.0,1216.0,980.0,0.0,0.0,692.0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258582,1,73.0,71.0,3550.0,3335.0,0.0,0.0,582.0,0,0,...,0,0,0,0,0,0,0,0,0,0
258583,1,20.0,19.0,1444.0,1255.0,0.0,0.0,582.0,0,0,...,0,0,0,0,0,0,0,0,0,0
258584,1,43.0,43.0,2150.0,1951.0,0.0,0.0,553.0,0,0,...,0,0,0,0,0,0,0,0,0,0
258585,1,31.0,31.0,1794.0,1598.0,0.0,0.0,553.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
# cols = ['avg_dep_delay', 'departures_scheduled', 'departures_performed', 'seats', 'passengers', 'freight', 'mail', 'distance']
# numeric = full[cols].copy()
# numeric = pd.DataFrame(minmax.fit_transform(numeric), columns=cols)
# full.drop(columns=cols, inplace=True)
# full.join(numeric)
# full.drop(columns=cols)

In [31]:
cols = ['departures_scheduled', 'departures_performed', 'seats', 'passengers', 'freight', 'mail', 'distance']
cols_scaled = [s + "_scaled" for s in cols]
minmax = MinMaxScaler()
full[cols_scaled] = minmax.fit_transform(full[cols])

In [32]:
full

,avg_arr_delay,departures_scheduled,departures_performed,seats,passengers,freight,mail,distance,Month_1,Month_2,...,dest_airport_id_16133,dest_airport_id_16218,dest_airport_id_16869,departures_scheduled_scaled,departures_performed_scaled,seats_scaled,passengers_scaled,freight_scaled,mail_scaled,distance_scaled
0,1,29.0,29.0,3190.0,2475.0,0.0,0.0,692.0,1,0,...,0,0,0,0.031183,0.032520,0.028586,0.025841,0.000000,0.0,0.133097
1,0,36.0,35.0,2660.0,1998.0,0.0,0.0,692.0,1,0,...,0,0,0,0.038710,0.039489,0.023781,0.020861,0.000000,0.0,0.133097
2,1,41.0,40.0,3040.0,2492.0,0.0,0.0,692.0,1,0,...,0,0,0,0.044086,0.045296,0.027226,0.026019,0.000000,0.0,0.133097
3,0,26.0,26.0,2860.0,2422.0,71.0,0.0,692.0,1,0,...,0,0,0,0.027957,0.029036,0.025594,0.025288,0.000029,0.0,0.133097
4,1,16.0,16.0,1216.0,980.0,0.0,0.0,692.0,0,1,...,0,0,0,0.017204,0.017422,0.010689,0.010232,0.000000,0.0,0.133097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258582,1,73.0,71.0,3550.0,3335.0,0.0,0.0,582.0,0,0,...,0,0,0,0.078495,0.081301,0.031850,0.034820,0.000000,0.0,0.111439
258583,1,20.0,19.0,1444.0,1255.0,0.0,0.0,582.0,0,0,...,0,0,0,0.021505,0.020906,0.012756,0.013103,0.000000,0.0,0.111439
258584,1,43.0,43.0,2150.0,1951.0,0.0,0.0,553.0,0,0,...,0,0,0,0.046237,0.048780,0.019157,0.020370,0.000000,0.0,0.105729
258585,1,31.0,31.0,1794.0,1598.0,0.0,0.0,553.0,0,0,...,0,0,0,0.033333,0.034843,0.015929,0.016685,0.000000,0.0,0.105729


In [33]:
full = full.drop(columns=['departures_scheduled', 'departures_performed', 'seats', 'passengers', 'freight', 'mail', 'distance'])

In [37]:
full

,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,...,dest_airport_id_16218,dest_airport_id_16869,departures_scheduled_scaled,departures_performed_scaled,seats_scaled,passengers_scaled,freight_scaled,mail_scaled,distance_scaled,avg_arr_delay
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0.031183,0.032520,0.028586,0.025841,0.000000,0.0,0.133097,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0.038710,0.039489,0.023781,0.020861,0.000000,0.0,0.133097,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0.044086,0.045296,0.027226,0.026019,0.000000,0.0,0.133097,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0.027957,0.029036,0.025594,0.025288,0.000029,0.0,0.133097,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0.017204,0.017422,0.010689,0.010232,0.000000,0.0,0.133097,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258582,0,0,0,0,0,0,0,0,0,0,...,0,0,0.078495,0.081301,0.031850,0.034820,0.000000,0.0,0.111439,1
258583,0,0,0,0,0,0,0,0,0,0,...,0,0,0.021505,0.020906,0.012756,0.013103,0.000000,0.0,0.111439,1
258584,0,0,0,0,0,0,0,0,0,1,...,0,0,0.046237,0.048780,0.019157,0.020370,0.000000,0.0,0.105729,1
258585,0,0,0,0,0,0,0,0,0,0,...,0,0,0.033333,0.034843,0.015929,0.016685,0.000000,0.0,0.105729,1


In [39]:
full.to_csv("dfV3ENC.csv", index=False)